In [2]:
%load_ext cython

In [4]:
%%cython

cdef f():
    return 1

def f1():
    return f()+f()

print('hello', f1())

hello 2


In [3]:
f1()

2

bint is a cython type, neither a Python or a C type

# Additions to Python's function syntax

In [35]:
%%cython

cpdef int func1(int x, int y):
    if x+y == 45:
        raise Exception("error", x, y)
    return x

cpdef int func2(int x, int y) except 45:
    if x+y == 40 or x+y == 45:
        raise Exception("error", x, y)
    return x

cpdef int func3(int x, int y) except? 45:
    if x+y == 45:
        raise Exception("error", x, y)
    return x

In [36]:
func1(45, 0)

Exception ignored in: '_cython_magic_b7e1e2aeb327ddd9c4b5318d0e87dda2.func1'
Exception: ('error', 45, 0)


0

In [46]:
try:
    func2(45, 0)
except Exception as e:
    print(e)
print('---')
try:
    func2(39, 1)
except Exception as e:
    print(e)
print('---')
print(func2(39, 0))
print('---')
try:
    func2(45, 1)
except Exception as e:
    print(e)

('error', 45, 0)
---
('error', 39, 1)
---
39
---
<built-in function func2> returned NULL without setting an error


In [24]:
func(45, -1)

45

In [25]:
func(44, 1)

Exception: (44, 1)

# Cython `cdef` classes
(also called **extension types**)

In [7]:
%%cython

cdef class CyAnimal:
    cdef int legs
    cdef int tails
    def __cinit__(self, legs, tails): # __init__ would also work
        self.legs = legs
        self.tails = tails
    
    cdef int limbs1(CyAnimal self):
        return self.legs * self.tails
    
    cpdef int limbs2(CyAnimal self):
        return self.limbs1() + self.legs
    
    def limbs3(self):
        return self.limbs1() + self.tails
    
    def limbs4(CyAnimal self):
        return self.limbs1() + self.tails + 100
    
    
c1 = CyAnimal(4,5)
try:
    c1.limbs1()
except AttributeError as e:
    print(e)
print(c1.limbs2(), c1.limbs3(), c1.limbs4())

'_cython_magic_0351e041d1b014c87262d0addcf52c0f.CyA' object has no attribute 'limbs1'
24 25 125


In [9]:
dir(c1)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'limbs2',
 'limbs3',
 'limbs4']

In [10]:
type(c1)

_cython_magic_0351e041d1b014c87262d0addcf52c0f.CyAnimal

In [11]:
c1.legs1 = 12

AttributeError: '_cython_magic_0351e041d1b014c87262d0addcf52c0f.CyAnimal' object has no attribute 'legs1'

In [13]:
CyAnimal.__init__

<slot wrapper '__init__' of 'object' objects>

In [14]:
c1.__init__

<method-wrapper '__init__' of _cython_magic_0351e041d1b014c87262d0addcf52c0f.CyAnimal object at 0x7f9a580c4fb0>

In [15]:
c1.__cinit__

AttributeError: '_cython_magic_0351e041d1b014c87262d0addcf52c0f.CyA' object has no attribute '__cinit__'

# `cpdef` class?

In [26]:
%%cython

cpdef class Hello:
    pass


Error compiling Cython file:
------------------------------------------------------------
...

cpdef class Hello:
     ^
------------------------------------------------------------

/home/sourav/.cache/ipython/cython/_cython_magic_d73a7492d9abb718c47994dc55bc5a21.pyx:2:6: Extension types cannot be declared cpdef



# `cdef`/`cpdef` closures?

In [19]:
%%cython
cdef gen_hello(str who): # NOTE that I have not specified the return type
    def hello(): # inner `cdef` function not allowed by compiler
        print(who)
    return hello

func1 = gen_hello("world")
func1()

world


In [20]:
%%cython
def gen_bye(who): # does not work
    cdef say_bye():
        print(who)
    return say_bye

func2 = gen_bye("world")
func2()


Error compiling Cython file:
------------------------------------------------------------
...
def gen_bye(who): # does not work
    cdef say_bye():
                 ^
------------------------------------------------------------

/home/sourav/.cache/ipython/cython/_cython_magic_bca423abe276fb503bcde794ec253b00.pyx:2:18: C function definition not allowed here


# Inheritance

In [8]:
%%cython

cdef class CyList(list):
    def __init__(self, list arg):
        super().__init__(arg)
    
    def __repr__(self):
        return '%s(%s)' % (self.__class__.__name__, super().__repr__())

In [9]:
clst = CyList([8])

In [10]:
clst

CyList([8])

In [11]:
type(clst)

_cython_magic_67b4c6d39830656463c064d9ccb059d9.CyList

In [41]:
%%cython

class Aclass(list):
    pass

cdef class Bclass(Aclass): # Not allowed
    pass


Error compiling Cython file:
------------------------------------------------------------
...

class Aclass(list):
    pass

cdef class Bclass(Aclass): # Not allowed
    ^
------------------------------------------------------------

/home/sourav/.cache/ipython/cython/_cython_magic_c73d6556b4b25aaa97fb2428f42db343.pyx:5:5: 'Aclass' is not a type name


In [37]:
%%cython
cdef class Oclass(object): # no problem
    pass

In [46]:
%%cython

class Aclass:
    cdef method(self): # not allowed here
        pass


Error compiling Cython file:
------------------------------------------------------------
...

class Aclass:
    cdef method(self): # not allowed here
        ^
------------------------------------------------------------

/home/sourav/.cache/ipython/cython/_cython_magic_5fef62fcde35fe47610c27b9ccd7903b.pyx:3:9: cdef statement not allowed here


In [50]:
%%cython

class Aclass:
    pass

cdef method(self, int x):
    print("inside %s, x=%s" % (type(self), x))

Aclass.method = method # no problem
Aclass().method = method # no problem

In [51]:
Aclass().method(55)

inside <class '_cython_magic_efa9c83ed080fbc7b6dfa554cf0b6273.Aclass'>, x=55


# Instance intialization and calling `super`

In [11]:
%%cython

cdef class Xclass:
    cdef int x1, x2
    def __cinit__(self, x):
        self.x1 = self.x2 = x
        self.x2 += 1

def repr(self):
    return '%s(%s, %s)' % (type(self).__name__, self.x1, self.x2)

try:
    Xclass.__repr__ = repr
except TypeError as e:
    print(e)

can't set attributes of built-in/extension type '_cython_magic_42ebc38c32f7007b4255fb69a840015b.Xclass'


In [15]:
%%cython

cdef class Xclass:
    cdef int x1, x2
    def __cinit__(self, x):
        self.x1 = self.x2 = x
        self.x1 += 1
    def __repr__(self):
        # return '%s(%s, %s)' % (__class__.__name__, self.x1, self.x2) # __class__ does not give the correct result
        assert __class__ is not type(self)
        return '%s(%s, %s)' % (type(self).__name__, self.x1, self.x2)

cdef class Yclass(Xclass):
    cdef int x3
    def __cinit__(self, x): # must have the same number of arguments as the inherited extension type's __cinit__
        #super(Yclass, self).__cinit__(x)  # super.__cinit__ not required and if called, raises an AttributeError
        self.x3 = x+2
    def __repr__(self):
        s = super().__repr__()
        s = s.replace(')', ', %s)' % self.x3)
        return s

obj1 = Xclass(3)
obj2 = Yclass(10)
print(obj1, obj2)

Xclass(4, 3) Yclass(11, 10, 12)


In [17]:
try:
    obj1.x1 = 5
except AttributeError as e:
    print(e)

'_cython_magic_e362d8a47fb371186f06e2af87efd113.Xclass' object has no attribute 'x1'


In [18]:
%%cython

cdef class Yclass(Xclass):
    cdef int x3
    def __cinit__(self, x, x3):
        super().__cinit__(x)
        self.x3 = x3


Error compiling Cython file:
------------------------------------------------------------
...

cdef class Yclass(Xclass):
    ^
------------------------------------------------------------

/home/sourav/.cache/ipython/cython/_cython_magic_1f291b65b6ce6020bcc14529416008a1.pyx:2:5: 'Xclass' is not declared


In [4]:
Xclass

_cython_magic_bf59dedea835732d306aa0a389c7c93c.Xclass

# Type casting

In [44]:
%%cython

cdef class A:
    cdef int x
    def __cinit__(self, x):
        self.x = x

def afunc(int i, A a):
    print("tear", i, a.x)

def bfunc(int i, A a not None):
    print("here", i, a.x)


try:
    afunc(1, 8)
except TypeError as e:
    print(1, e)

a1 = A(99)
afunc(2, a1)

try:
    bfunc(3, None)
except TypeError as e:
    print(3, e)

try:
    x = None
    afunc(4, <A?>x)
except TypeError as e:
    print(4, e)

afunc(5, None) # segfault or wrong answer?

1 Argument 'a' has incorrect type (expected _cython_magic_2bc488e49c9465c5305b68ba1f4560d8.A, got int)
tear 2 99
3 Argument 'a' has incorrect type (expected _cython_magic_2bc488e49c9465c5305b68ba1f4560d8.A, got NoneType)
4 Cannot convert NoneType to _cython_magic_2bc488e49c9465c5305b68ba1f4560d8.A
tear 5 0


In [46]:
bfunc(1, a1)
try:
    bfunc(2, None)
except TypeError as e:
    print(e)

here 1 99
Argument 'a' has incorrect type (expected _cython_magic_2bc488e49c9465c5305b68ba1f4560d8.A, got NoneType)


# Polymorphism

# Assigning values to typed variables

In [2]:
%%cython -2

cdef list lst
try:
    lst = range(4)
except TypeError as e:
    print(e)
else:
    print lst


Expected list, got range


In [3]:
%%cython -3

cdef list lst
try:
    lst = range(4)
except TypeError as e:
    print(e)
else:
    print(lst)


Expected list, got range


# Explain these

In [4]:
%%cython

cdef class Pclass:
    pass

cdef class Qclass:
    cdef int x, y

print(sizeof(Pclass), sizeof(Pclass()))
print(sizeof(Qclass), sizeof(Qclass()))

16 8
24 8


# Gotchas

## `NameError` in `%%cython` cells
A common beginner error is to attempt to access a Cython variable defined in one `%%cython` cell in an IPython terminal/notebook in another `%%cython` cell. This throws a `NameError` because the latter `%%cython` cell has no knowledge of the previous `%%cython` cell. On the other hand, in a normal cell all the (Python-level) names introduced by the latest executed `%%cython` cell are available.

In [32]:
%%cython
cpdef int var1=10, var2=20
var3 = 5
cdef class CyX:
    pass

In [33]:
var3, CyX # OK

(5, _cython_magic_d2bf0ea6c4588942d7a5ad6c7a9b7014.CyX)

In [34]:
var1, var2 # even `cpdef` vars throw NameError!

NameError: name 'var1' is not defined

In [35]:
%%cython
var4 = 4

In [39]:
var3, var4 # var3 belongs to the previous to previous %%cython cell, and var4 to the previous %%cython cell.
           # This works fine.

(5, 4)

### Conclusion
This means that `cdef` and `cpdef` vars are not exported outside a cell, but normal Python vars and extension types are exported.